In [1]:
import os
import json
import minsearch
from dotenv import load_dotenv

# Load .env file
load_dotenv()

API_KEY = os.getenv("API_KEY")

## Prepare QA document

In [2]:
with open("documents.json", "rt") as f:
    docs_raw = json.load(f)

In [3]:
documents = []

for docs_dict in docs_raw:
    for doc in docs_dict["documents"]:
        doc["course"] = docs_dict["course"]
        documents.append(doc)

In [4]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

## MinSearch (Toy search)

In [5]:
index = minsearch.Index(
    text_fields=["question", "text", "section"], keyword_fields=["course"]
)

In [6]:
q = "the course has already started, can I still enroll?"

In [7]:
index.fit(documents)

## OpenAI API usage

In [8]:
from openai import OpenAI

client = OpenAI(api_key=API_KEY)

In [9]:
response = client.chat.completions.create(
    model="gpt-4o", messages=[{"role": "user", "content": q}]
)

response.choices[0].message.content

"If a course has already started, your ability to enroll may depend on several factors, including the policies of the institution offering the course and the specific requirements of the course itself. Here are a few steps you can take to determine whether late enrollment is possible:\n\n1. **Check the Course Policy:** Look for any information provided by the institution or on the course's webpage regarding late enrollment. Some courses might have a grace period during which late enrollments are accepted.\n\n2. **Contact the Instructor:** Reach out directly to the course instructor or professor to inquire if they are willing to make exceptions for late enrollments. Be sure to explain your situation and demonstrate your eagerness to catch up with the course material.\n\n3. **Contact the Registrar or Academic Advisor:** The registrar's office or an academic advisor at the institution may have more detailed information on the official policies and may be able to assist you in enrolling la

## Search query from documents using MinSearch

In [10]:
def search(query):
    boost = {"question": 3.0, "section": 0.5}

    results = index.search(
        query=q,
        filter_dict={"course": "data-engineering-zoomcamp"},
        boost_dict=boost,
        num_results=5,
    )

    return results

In [11]:
def build_prompt(query, search_results):
    prompt_template = """
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
    Use only the facts from the CONTEXT when answering the QUESTION.

    QUESTION: {question}

    CONTEXT: 
    {context}
    """.strip()

    context = ""

    for doc in search_results:
        context = (
            context
            + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
        )

    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [12]:
def gpt(prompt):
    response = client.chat.completions.create(
        model="gpt-4o", messages=[{"role": "user", "content": prompt}]
    )

    return response.choices[0].message.content

In [13]:
query = "how do I run kafka?"


def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = gpt(prompt)

    return answer

In [14]:
rag(query)

"The provided CONTEXT does not include specific information on how to run Kafka. Therefore, I'm unable to give you detailed instructions on running Kafka based on the provided CONTEXT. For more specific guidance, I recommend checking the course materials related to Kafka or asking on the Slack channel mentioned for additional technical support."

In [15]:
rag("the course has already started, can I still enroll?")

'Yes, you can still enroll in the course even after it has started. You are still eligible to submit the homeworks. However, be mindful of the deadlines for turning in the final projects; make sure not to leave everything until the last minute.'

## Elastic Search

In [16]:
from elasticsearch import Elasticsearch

In [17]:
es_client = Elasticsearch("http://localhost:9200")

In [18]:
es_client.info()

ObjectApiResponse({'name': '13e86c65ae14', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'E_wQiibfS6C7pPTjleAamQ', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [19]:
index_settings = {
    "settings": {"number_of_shards": 1, "number_of_replicas": 0},
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"},
        }
    },
}

In [20]:
index_name = "course-questions"
es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [21]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [22]:
from tqdm.auto import tqdm

In [23]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

In [24]:
query = "I just discovered the course. Can I still join it?"

In [25]:
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^3", "text", "section"],
                    "type": "best_fields",
                }
            },
            "filter": {"term": {"course": "data-engineering-zoomcamp"}},
        }
    },
}

In [26]:
response = es_client.search(index=index_name, body=search_query)

In [27]:
result_docs = []

for hit in response["hits"]["hits"]:
    result_docs.append(hit["_source"])

In [28]:
result_docs

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (insta